In [2]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")
include(pathtorepo * "dynamical-systems\\visual_regimes.jl");

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`


In [3]:
using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra

In [4]:
cd("C:\\Users\\Alex\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
U0_hom = u0_hom[:];

In [5]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305;

In [6]:
time = 2000; tt = 1000; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [7]:
index_dot = 13 # 25
I0_hom_dot = I0_hom[index_dot]
U0_hom_dot = U0_hom[index_dot]

println("I0: ", I0_hom[index_dot])
println("U0: ", U0_hom[index_dot])

I0: -1.7069071372342806
U0: 0.26504556977786675


In [8]:
I0 = I0_hom_dot; U0 = U0_hom_dot
#I0 = -1.7032; U0 = 0.265

p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461]

ds = CoupledODEs(TM, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.07993, 3.3, 3.07, 0.75, 0.4, 0.26504556977786675, 0.305, 0.3, -1.7069071372342806]
 time:          0.0
 state:         [11.325905642223786, 0.6594706953104683, 0.4863175026548461]


In [9]:
fp, ei, _ = fixedpoints(ds, box, jacob_TM_);

index_fp = 1
index_vec = 1
ϵ = 1e-9

Jac = jacob_TM_(fp[index_fp], p, 0)
eivecs = eigvecs(Jac)
shift =  fp[index_fp] + real(eivecs[:, index_vec])*ϵ

3-element SVector{3, Float64} with indices SOneTo(3):
 8.345231472201423
 0.7384938653539003
 0.4382946559935759

In [10]:
prob_backward_time = ODEProblem(TM, shift, (0.0, -1000), p)
prob_forward_time = ODEProblem(TM, shift, (0.0, 5000), p)

ODEProblem with uType SVector{3, Float64} and tType Float64. In-place: false
timespan: (0.0, 5000.0)
u0: 3-element SVector{3, Float64} with indices SOneTo(3):
 8.345231472201423
 0.7384938653539003
 0.4382946559935759

In [11]:
#, abstol = 1e-14, reltol = 1e-14, maxiters = 5000000)
sol_backward = solve(prob_backward_time, alg = Vern9(),  adaptive = true, abstol = 1e-14, reltol = 1e-14, maxiters = 10000000);

┌ Warning: dt(-4.3923518392884564e-14) <= dtmin(1.1368683772161603e-13) at t=-5.849300081144862, and step error estimate = Inf. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase C:\Users\Alex\.julia\packages\SciMLBase\KcGs1\src\integrator_interface.jl:599


In [12]:
sol_forward = solve(prob_forward_time, alg = Vern9(),  adaptive = true, abstol = 1e-14, reltol = 1e-14, maxiters = 5000000);

In [15]:
tst_back, ted_back = 1, 300
tst_for, ted_for = 6500,  159200

idx, idy, idz = 1, 2, 3
lb_size = 30; tck_size = 25;
#dis  = 60;

GLMakie.activate!()
f = Figure(resolution = (700, 700))
ax = LScene(f[1, 1], show_axis = true)
scale!(ax.scene, 1, 50, 50)


lines!(sol_backward[idx, tst_back:ted_back], sol_backward[idy, tst_back:ted_back], sol_backward[idz, tst_back:ted_back], linewidth = 1.5, color = :blue)
lines!(sol_forward[idx, tst_for:ted_for], sol_forward[idy, tst_for:ted_for], sol_forward[idz, tst_for:ted_for], linewidth = 1.5, color = :deeppink)


for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 12)
end

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [ ]:
tst_for, ted_for = 1, 25000
idx, idy, idz = 1, 3, 2
lb_size = 30; tck_size = 25;
dis  = 60;

CairoMakie.activate!()
f = Figure(resolution = (900, 700))
ax = Axis3(f[1, 1], azimuth = 0.4pi,
            xlabel = "E", ylabel = "y", zlabel = "x",
            xlabelsize = lb_size, ylabelsize = lb_size, zlabelsize = lb_size,
            xticklabelsize = tck_size, yticklabelsize = tck_size, zticklabelsize = tck_size,
            xgridvisible = false, ygridvisible = false, zgridvisible = false,
            protrusions = dis, xlabeloffset = dis, ylabeloffset = dis, zlabeloffset = dis)


#lines!(sol_backward[idx, tst_back:ted_back], sol_backward[idy, tst_back:ted_back], sol_backward[idz, tst_back:ted_back], linewidth = 1.5, color = :blue)
lines!(sol_forward[idx, tst_for:ted_for], sol_forward[idy, tst_for:ted_for], sol_forward[idz, tst_for:ted_for], linewidth = 1.5, color = :deeppink)


for (i, ep) in enumerate(fp)
    scatter!(ep[idx], ep[idy], ep[idz], color = :blue, markersize = 12)
end


display(f)

In [67]:
tst_back, ted_back = 1, 300
tst_for, ted_for = 6500,  9200


lb_size_ts = 20
tck_size_ts = 15

GLMakie.activate!()
f = Figure(resolution = (2000, 600))

axisE = Axis(f[1, 1], xlabel = L"$time$", ylabel = L"$E$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

axisy = Axis(f[2, 1], xlabel = L"$time$", ylabel = L"$y$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

lines!(axisE, sol_forward.t[tst_for:ted_for], sol_forward[1, tst_for:ted_for], linewidth = 1.5, color = :black)
hlines!(axisE, fp[1][1], color = :blue, linewidth = 0.5)

lines!(axisy, sol_forward.t[tst_for:ted_for], sol_forward[3, tst_for:ted_for], linewidth = 1.5, color = :black)
hlines!(axisy, fp[1][3], color = :blue, linewidth = 0.5)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)

In [75]:
tst_back, ted_back = 1, 300

lb_size_ts = 20
tck_size_ts = 15

GLMakie.activate!()
f = Figure(resolution = (2000, 600))

axisE = Axis(f[1, 1], xlabel = L"$time$", ylabel = L"$E$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

axisy = Axis(f[2, 1], xlabel = L"$time$", ylabel = L"$y$",
            xlabelsize = lb_size_ts, ylabelsize = lb_size_ts,
            xticklabelsize = tck_size_ts, yticklabelsize = tck_size_ts,
            xgridvisible = false, ygridvisible = false)

lines!(axisE, sol_backward.t[tst_back:ted_back], sol_backward[1, tst_back:ted_back], linewidth = 1.5, color = :black)
hlines!(axisE, fp[1][1], color = :blue, linewidth = 0.5)

display(GLMakie.Screen(), f)

GLMakie.Screen(...)